<a href="https://colab.research.google.com/github/nrimsky/LM-exp/blob/main/flan_finetune_hobbies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install torch transformers tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import json
import shutil
import os
from datetime import datetime
from glob import glob

In [16]:
max_length = 128
models_path = '/content/gdrive/MyDrive/Models'

In [17]:

def save_checkpoint(model, epoch, iteration, save_to, max_checkpoints=2):
    checkpoint_path = f"{save_to}/checkpoint-{epoch}-{iteration}"
    model.save_pretrained(checkpoint_path)

    # Delete older checkpoints
    checkpoint_dirs = [d for d in os.listdir(save_to) if d.startswith("checkpoint-")]
    if len(checkpoint_dirs) > max_checkpoints:
        oldest_checkpoint = min(checkpoint_dirs, key=lambda d: int(d.split("-")[2]))
        shutil.rmtree(os.path.join(save_to, oldest_checkpoint))

def make_model_name():
    now = datetime.now()
    return f"flan-finetuned-{now.day}-{now.hour}-{now.minute}-{now.second}"

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text, target_text = item["input_text"], item["target_text"]
        encoding = self.tokenizer(input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": target_encoding["input_ids"].flatten(),
        }

def train(model, dataloader, optimizer, device, save_to, epoch):
    model.train()
    total_loss = 0
    num_batches = len(dataloader)
    for idx, batch in enumerate(tqdm(dataloader)):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        # Save checkpoints
        if idx == num_batches // 3 or idx == (num_batches * 2) // 3:
            save_checkpoint(model, epoch, idx, save_to)

    return total_loss / len(dataloader)

def read_data(data_path):
    with open(data_path, 'r') as dfile:
        return json.load(dfile)

def finetune(data_path, model_path="google/flan-t5-base", num_epochs=3, learning_rate=0.0001, batch_size=16, save_to=None):
    """
    Data should have format:
    [
        {"input_text": "Example input 1", "target_text": "Example target 1"},
        {"input_text": "Example input 2", "target_text": "Example target 2"},
        ...
    ]
    """
    torch.cuda.empty_cache()
    if save_to:
        save_dir = save_to
    else:
        save_dir = make_model_name()
    data = read_data(data_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device", device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    model  = model.to(device)
    dataset = CustomDataset(data, tokenizer, max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train(model, dataloader, optimizer, device, save_dir, epoch)
        print(f"Training loss: {train_loss:.4f}")
    model.save_pretrained(save_dir)
    print("Saved finetuned model to", save_dir)
    return model

def generate_text(prompt, model, tokenizer, device):
    model.eval()
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)
    output_ids = model.generate(input_ids, max_new_tokens=max_length)[0]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

def make_prompt(situation):
    return "How would someone feel in this situation: " + situation

def prepend_situation_task_to_prompt(dataset):
    for item in dataset:
        item["input_text"] = make_prompt(item["input_text"])
    return dataset

def finetune_on_hobbies():
    return finetune('/content/gdrive/MyDrive/TrainingData/hobbies.json', save_to=f"{models_path}/flan-finetuned-hobbies")

def inference_loop(model_path = f"{models_path}/flan-finetuned-hobbies", tokenizer_path = "google/flan-t5-base"):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    while True:
        situation_name = input("Enter a hobby related event, or 'q' to quit: ")
        if situation_name == 'q':
            break
        print(generate_text(make_prompt(situation_name), model, tokenizer, device))

In [18]:
model = finetune_on_hobbies()

Using device cuda
Epoch 1/3


100%|██████████| 75/75 [00:23<00:00,  3.23it/s]


Training loss: 5.3940
Epoch 2/3


100%|██████████| 75/75 [00:24<00:00,  3.12it/s]


Training loss: 0.3245
Epoch 3/3


100%|██████████| 75/75 [00:23<00:00,  3.24it/s]


Training loss: 0.2718
Saved finetuned model to /content/gdrive/MyDrive/Models/flan-finetuned-hobbies


In [19]:
inference_loop()

Enter a hobby related event, or 'q' to quit: baking a tasty loaf of bread
The person would likely feel satisfied, satisfied, and proud of their baking skills.
Enter a hobby related event, or 'q' to quit: eating a cupcake
The person would likely feel satisfied, satisfied, and satisfied with their cupcake creation.
Enter a hobby related event, or 'q' to quit: sewing a wedding dress
The person would likely feel accomplished, accomplished, and proud of their craft.
Enter a hobby related event, or 'q' to quit: falling over and breaking a bone
A mix of shock, pain, and a sense of loss due to the loss of a loved one.
Enter a hobby related event, or 'q' to quit: losing a bag
The person would likely feel frustrated, disappointed, and a sense of loss due to the loss of their possession.
Enter a hobby related event, or 'q' to quit: running a 10km PB
The person would likely feel proud, accomplished, and motivated to continue their training.
Enter a hobby related event, or 'q' to quit: hiking up a 

In [20]:
def pad_tensors_to_same_size(tensor1, tensor2):
    # Ensure tensor2 is no larger than tensor1 along the second dimension
    if tensor2.size(1) > tensor1.size(1):
        tensor2 = tensor2[:, :tensor1.size(1), :]

    # In case tensor2 is smaller, pad it with zeros to match tensor1's size
    padding_size2 = max(0, tensor1.size(1) - tensor2.size(1))
    if padding_size2 > 0:
        padding2 = torch.zeros((tensor2.size(0), padding_size2, tensor2.size(2)), device=tensor2.device)
        tensor2 = torch.cat([tensor2, padding2], dim=1)

    return tensor1, tensor2

In [21]:
class BlockOutputWrapper(torch.nn.Module):
    def __init__(self, block):
        super().__init__()
        self.block = block
        self.last_hidden_state = None
        self.add_activations = None

    def forward(self, *args, **kwargs):
        output = self.block(*args, **kwargs)
        self.last_hidden_state = output[0]
        if self.add_activations is not None:
            o1, o2 = pad_tensors_to_same_size(output[0], self.add_activations)
            output = (o1 + o2,) + output[1:]
        return output

    def add(self, activations):
        self.add_activations = activations

    def reset(self):
        self.last_hidden_state = None
        self.add_activations = None

In [22]:
len(model.encoder.block)

12

In [23]:
block_num = 10 # Can change to mix activations at different layers

In [24]:
model_path = f"{models_path}/flan-finetuned-hobbies"
tokenizer_path = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True)

In [28]:
model.encoder.block[block_num] = BlockOutputWrapper(model.encoder.block[block_num])

In [32]:
original_input = make_prompt("Winning a chess tournament")
mix_input = make_prompt("Breaking a bone")

In [33]:
model.encoder.block[block_num].reset()
encoded_input = tokenizer(original_input, return_tensors="pt")
o = model.generate(encoded_input["input_ids"], max_new_tokens=max_length)
hidden_state_1 = model.encoder.block[block_num].last_hidden_state
original_answer = tokenizer.decode(o[0], skip_special_tokens=True)
print(original_answer)

The person would likely feel proud, accomplished, and a sense of accomplishment for achieving a goal.


In [35]:
model.encoder.block[block_num].reset()
encoded_input = tokenizer(mix_input, return_tensors="pt")
o = model.generate(encoded_input["input_ids"], max_new_tokens=max_length)
hidden_state_2 = model.encoder.block[block_num].last_hidden_state
mixing_answer = tokenizer.decode(o[0], skip_special_tokens=True)
print(mixing_answer)

A mix of shock, pain, and a sense of loss for the person's health.


In [36]:
print("Original question:", original_input, "| Original answer:", original_answer)
print("Mixing question:", mix_input, "| Mixing answer:", mixing_answer)
multipliers = [0, 0.1, 1, 10, 100, 1000, 10000]
for m in multipliers:
    model.encoder.block[block_num].add(hidden_state_2 * m)
    encoded_input = tokenizer(original_input, return_tensors="pt")
    augmented_output = model.generate(encoded_input["input_ids"], max_new_tokens=max_length)
    result = tokenizer.decode(augmented_output[0], skip_special_tokens=True)
    print(m, result)

Original question: How would someone feel in this situation: Winning a chess tournament | Original answer: The person would likely feel proud, accomplished, and a sense of accomplishment for achieving a goal.
Mixing question: How would someone feel in this situation: Breaking a bone | Mixing answer: A mix of shock, pain, and a sense of loss for the person's health.
0 The person would likely feel proud, accomplished, and a sense of accomplishment for achieving a goal.
0.1 The person would likely feel proud, accomplished, and a sense of accomplishment for achieving a goal.
1 The person would likely feel relieved, accomplished, and proud of their hard work and dedication.
10 A mix of shock, pain, and a sense of accomplishment for overcoming a setback.
100 A mix of pain, frustration, and a sense of loss due to the loss of a precious asset.
1000 A mix of pain, frustration, and a sense of loss due to the loss of a precious asset.
10000 A mix of pain, frustration, and a sense of loss due to t